In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
import matplotlib.pyplot as plt

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [ ]:
EPOCHS = 10000
BATCH_SIZE = 128

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        root = './.data',
        train = True,
        download = True,
        transform = transforms.Compose([
#             transforms.RandomCrop(32, padding=4)
            transforms.RandomCrop(32),
            transforms.RandomHorizontalFlip(),
            #0~1로 자동으로 normalize
            transforms.ToTensor(),
            #output[channel] = (input[channel] - mean[channel] ) / std[channel]
            #각 채널의 픽셀값(0)
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    ), 
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        root = './.data',
        train = False,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
        ])        
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8*8*64, 384)
        self.fc2 = nn.Linear(384, 10)
    
    def forward(self, x):
        h_conv1 = F.relu(self.conv1(x))
#         print(f'h_conv1 shape:{h_conv1.shape}')
        h_pool1 = F.max_pool2d(h_conv1, 3, stride=2, padding=1)
#         print(f'h_pool1 shape:{h_pool1.shape}')
        h_conv2 = F.relu(self.conv2(h_pool1))
#         print(f'h_conv2 shape:{h_conv2.shape}')
        h_pool2 = F.max_pool2d(h_conv2, 3, stride=2, padding=1)
#         print(f'h_pool2 shape:{h_pool2.shape}')
        h_conv3 = F.relu(h_pool2)
#         print(f'h_conv3 shape:{h_conv3.shape}')
        h_conv4 = F.relu(h_conv3)
#         print(f'h_conv4 shape:{h_conv4.shape}')
        h_conv5 = F.relu(h_conv4)
#         print(f'h_conv5 shape:{h_conv5.shape}')
        flatten = h_conv5.view(-1, 8*8*64)
        h_fc1 = F.relu(self.fc1(flatten))
        h_fc1_drop = F.dropout(h_fc1, p=0.2, training=self.training)
        logits = self.fc2(h_fc1_drop)
        
        return F.log_softmax(logits, dim=1)        

In [ ]:
model = CNN().to(DEVICE)
optimizer = optim.RMSprop(model.parameters(), lr=1e-3)

In [ ]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
#         print(f'output shape:{output.shape}')
#         print(f'target shape:{target.shape}')
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train epoch : {epoch} {batch_idx * len(data)}/{len(train_loader.dataset)} ({100.0 * batch_idx / len(train_loader):.0f}%)\tLoss:{loss.item():.6f}')


In [ ]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100 * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [ ]:
for epoch in range(1, EPOCHS+1):
    model.train()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(f'{epoch} Test Loss : {test_loss:.4f}, Accuracy : {test_accuracy:.2f}')
#     print(f'{epoch}')